# Importing libraries 

In [1]:
# Importing analysis libraries needed
import pandas as pd
import numpy as np
import os

In [2]:
# Turning project folder into string path 
path = r'C:\Users\keirr\11-2020 Instacart Basket Analysis'

In [3]:
# Importing customers.csv with path
df_customer = pd.read_csv(os.path.join(path, '02 Data', 'OG Data', 'customers.csv'))

In [4]:
# Checking imported df
df_customer.head(10)

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374
5,133128,Cynthia,Noble,Female,Kentucky,43,1/1/2017,2,married,49643
6,152052,Chris,Walton,Male,Montana,20,1/1/2017,0,single,61746
7,168851,Joseph,Hickman,Male,South Carolina,30,1/1/2017,0,single,63712
8,69965,Jeremy,Vang,Male,Texas,47,1/1/2017,1,married,162432
9,82820,Shawn,Chung,Male,Virginia,26,1/1/2017,2,married,32072


# Contents
## 1. Exploring dataset
## 2. Data Wrangling: Customers dataset
## 3. Consistency Checks: Customers dataset
## 4. Merging df_customer with orders_products_agg
## 5. Exporting merged datasets

# 1. Exploring dataset

In [8]:
# Additionally looking into shape of df
df_customer.shape

(206209, 10)

In [9]:
# Looking into basic statistics of df
df_customer.describe()

,user_id,Age,n_dependants,income
count,206209.000000,206209.000000,206209.000000,206209.000000
mean,103105.000000,49.501646,1.499823,94632.852548
std,59527.555167,18.480962,1.118433,42473.786988
min,1.000000,18.000000,0.000000,25903.000000
25%,51553.000000,33.000000,0.000000,59874.000000
50%,103105.000000,49.000000,1.000000,93547.000000
75%,154657.000000,66.000000,3.000000,124244.000000
max,206209.000000,81.000000,3.000000,593901.000000


### Insights: 'user_id' is more than likely being represented by integer, need to look into dtypes & possibly convert dtype to object 

In [10]:
# Checking data types
df_customer.dtypes

user_id          int64
First Name      object
Surnam          object
Gender          object
STATE           object
Age              int64
date_joined     object
n_dependants     int64
fam_status      object
income           int64
dtype: object

# 2. Data Wrangling: Customers dataset

In [11]:
# Converting 'user_id' col to string/object
df_customer['user_id'] = df_customer['user_id'].astype('str')

In [12]:
# Checking work
df_customer['user_id'].dtype

dtype('O')

In [13]:
# Calling df
df_customer.head(5)

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


### Insights: the naming convention is all off, compared to 'orders_products_agg', I want to make all column names consistent

In [15]:
# Renaming order_dow column
df_customer.rename(columns = {'First Name' : 'first_name', 'Surnam' : 'last_name', 'Gender' : 'gender', 'STATE' : 'state', 'Age' : 'age', 'n_dependants' : 'num_of_kids', 'fam_status' : 'married_single'}, inplace = True)

In [16]:
df_customer.head(5)

,user_id,first_name,last_name,gender,state,age,date_joined,num_of_kids,married_single,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


# 3. Consistency Checks: Customers dataset

## Searching for mix-type data

In [18]:
# Checking for mix-type data
for col in df_customer.columns.tolist():
  weird = (df_customer[[col]].map(type) != df_customer[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df_customer[weird]) > 0:
    print (col)

first_name


### Insights: 'first_name' column has mix type data, need to convert to string/object

In [19]:
# Addressing mix-type data 
df_customer['first_name'] = df_customer['first_name'].astype('str')

## Checking for missing values

In [20]:
# Checking for null values
df_customer.isnull().sum()

user_id           0
first_name        0
last_name         0
gender            0
state             0
age               0
date_joined       0
num_of_kids       0
married_single    0
income            0
dtype: int64

### Insights: No missing values 

## Checking for duplicates

In [21]:
# Subsetting df to isolate possible duplicates
df_dups = df_customer[df_customer.duplicated()]

In [22]:
# Calling subset
df_dups

,user_id,first_name,last_name,gender,state,age,date_joined,num_of_kids,married_single,income


### Insights: No duplicates

# 4. Merging df_customer with orders_products_agg

In [23]:
# Importing orders_products_agg.pkl dataset with path
ords_prods_agg = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_agg.pkl'))

In [24]:
# Checking imported df
ords_prods_agg.head(5)

,order_id,user_id,order_number,orders_day_of_the_week,order_hour_of_day,days_since_last_order,product_id,add_to_cart_order,reordered,product_name,...,price_range_loc,busiest_days,slowest_days,busiest_period_of_day,max_order,loyalty_flag,mean_price,spending_flag,median_last_order,frequency_flag
0,2539329,1,1,2,8,NaN,196,1,0,Soda,...,Mid-range product,NaN,NaN,Regularly busy,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
1,2398795,1,2,3,7,15.0,196,1,1,Soda,...,Mid-range product,NaN,Slowest days,Regularly busy,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
2,473747,1,3,3,12,21.0,196,1,1,Soda,...,Mid-range product,NaN,Slowest days,Regularly busy,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
3,2254736,1,4,4,7,29.0,196,1,1,Soda,...,Mid-range product,NaN,Slowest days,Least busy,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
4,431534,1,5,4,15,28.0,196,1,1,Soda,...,Mid-range product,NaN,Slowest days,Least busy,10,New customer,6.367797,Low spender,20.5,Non-frequent customer


### Insights: both datasets have 'user_id'

In [25]:
# Checking shape of ords_prods_agg
ords_prods_agg.shape

(32404859, 24)

In [26]:
# Checking shape of df_customer to determine how to combine
df_customer.shape

(206209, 10)

### Insights: df_customer is nowhere near same size as ords_prods_agg, but there are mutiple instances of same user_id, perhaps data will merge sucessfully with inner merge

In [28]:
# Checking dtype of user_id in ords_prods_agg
ords_prods_agg.dtypes

order_id                     int64
user_id                      int64
order_number                 int64
orders_day_of_the_week       int64
order_hour_of_day            int64
days_since_last_order      float64
product_id                   int64
add_to_cart_order            int64
reordered                    int64
product_name                object
aisle_id                     int64
department_id                int64
prices                     float64
_merge                    category
price_range_loc             object
busiest_days                object
slowest_days                object
busiest_period_of_day       object
max_order                    int64
loyalty_flag                object
mean_price                 float64
spending_flag               object
median_last_order          float64
frequency_flag              object
dtype: object

In [32]:
# Converting user_id in df_customer back to integer to match key in ords_prods_agg
df_customer['user_id'] = df_customer['user_id'].astype(int)

In [33]:
# Dropping '_merge' column for future merge
ords_prods_agg.drop('_merge', axis=1, inplace=True)

In [34]:
# Test merge without overwriting
pd.merge(ords_prods_agg,df_customer, on = ['user_id'], indicator = True)

,order_id,user_id,order_number,orders_day_of_the_week,order_hour_of_day,days_since_last_order,product_id,add_to_cart_order,reordered,product_name,...,first_name,last_name,gender,state,age,date_joined,num_of_kids,married_single,income,_merge
0,2539329,1,1,2,8,NaN,196,1,0,Soda,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
1,2398795,1,2,3,7,15.0,196,1,1,Soda,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
2,473747,1,3,3,12,21.0,196,1,1,Soda,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
3,2254736,1,4,4,7,29.0,196,1,1,Soda,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
4,431534,1,5,4,15,28.0,196,1,1,Soda,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32404854,156685,106143,26,4,23,5.0,19675,1,1,Organic Raspberry Black Tea,...,Gerald,Yates,Male,Hawaii,25,5/26/2017,0,single,53755,both
32404855,484769,66343,1,6,11,NaN,47210,1,0,Fresh Farmed Tilapia Fillet,...,Jacqueline,Arroyo,Female,Tennessee,22,9/12/2017,3,married,46151,both
32404856,1561557,66343,2,1,11,30.0,47210,1,1,Fresh Farmed Tilapia Fillet,...,Jacqueline,Arroyo,Female,Tennessee,22,9/12/2017,3,married,46151,both
32404857,276317,66343,3,6,15,19.0,47210,1,1,Fresh Farmed Tilapia Fillet,...,Jacqueline,Arroyo,Female,Tennessee,22,9/12/2017,3,married,46151,both


### Insights: after running a test merge, how = inner, merge looks sucessful, multiples of user_id instances matched 

In [35]:
# Merging dataframes after test
ords_prods_customer = ords_prods_agg.merge(df_customer, on = ['user_id'], indicator = True)

In [38]:
# Calling df
ords_prods_customer.head(15)

,order_id,user_id,order_number,orders_day_of_the_week,order_hour_of_day,days_since_last_order,product_id,add_to_cart_order,reordered,product_name,...,first_name,last_name,gender,state,age,date_joined,num_of_kids,married_single,income,_merge
0,2539329,1,1,2,8,NaN,196,1,0,Soda,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
1,2398795,1,2,3,7,15.0,196,1,1,Soda,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
2,473747,1,3,3,12,21.0,196,1,1,Soda,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
3,2254736,1,4,4,7,29.0,196,1,1,Soda,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
4,431534,1,5,4,15,28.0,196,1,1,Soda,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
5,3367565,1,6,2,7,19.0,196,1,1,Soda,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
6,550135,1,7,1,9,20.0,196,1,1,Soda,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
7,3108588,1,8,1,14,14.0,196,2,1,Soda,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
8,2295261,1,9,1,16,0.0,196,4,1,Soda,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
9,2550362,1,10,4,8,30.0,196,1,1,Soda,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both


In [37]:
# Checking _merge flag
ords_prods_customer['_merge'].value_counts()

_merge
both          32404859
left_only            0
right_only           0
Name: count, dtype: int64

# 5. Exporting merged df

In [39]:
# Exporting combined datasets to pkl
ords_prods_customer.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'orders_products_customer.pkl'))